In [1]:
import pandas as pd
from os.path import join, isfile
from os import listdir
import numpy as np

In [3]:
datafiles = [join('.\\data', f) for f in listdir('./data') if join('./data', f).endswith(".csv")]

In [10]:
counter = 0
learning_frames = []
recall_frames = []
IDs = []
triggers = []
for f in datafiles[1:]:
    
    if "end_part2_text.started" in pd.read_csv(f):
        df = pd.read_csv(f, converters={'PID': str})
        IDs.append(df["PID"][0])
        triggers = df["wait_for_trigger.stopped"][df["starting_soon.started"].notna()].values
        df["recalled"] = 0

        for row in df.loc[df["test_trials.thisIndex"] >= 0].iterrows():
            recalled = row[1]["recall_reached.keys"]
            cue =  row[1]["Cue"]
            if recalled == 3:
                df.loc[df["Cue"] == cue, "recalled"] = 1
            else:
                df.loc[df["Cue"] == cue, "recalled"] = 0

        # LEARNING
        learning = df.loc[df["learning_trials.thisIndex"] >= 0]
        learning_frames.append(learning)

        
        # RECALL
        recall = df.loc[df["test_trials.thisIndex"] >= 0]
        recall_frames.append(recall)
        counter += 1
print('Number of finished experiments', counter)

Number of finished experiments 1


In [12]:
triggers

array([ 234.3803907,  810.8481725, 1547.923954 , 2114.8903387])

In [28]:
frame["recalled"]==1

11    False
12    False
13    False
14    False
15    False
      ...  
86    False
87    False
88    False
89    False
90    False
Name: recalled, Length: 80, dtype: bool

In [26]:
delim = ' '
form = '%.5f'
for frame in learning_frames:
    PID = frame["PID"].values[0]
    guess_times1 = frame.loc[frame["trial_type"]=="Guess","guess.started"] - triggers[0]
    guess_times2 = frame.loc[frame["trial_type"]=="Guess","guess.started"] - triggers[1]
    guess_times1 = guess_times1.values[guess_times2.values < 0]
    guess_times2 = guess_times2.values[guess_times2.values >= 0]
    guess_times1 = np.array([guess_times1, np.ones(len(guess_times1)) * 3, np.ones(len(guess_times1))]).T
    guess_times2 = np.array([guess_times2, np.ones(len(guess_times2)) * 3, np.ones(len(guess_times2))]).T
    np.savetxt('onset_files/' + PID + '_guess_onset_run1.txt', guess_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_guess_onset_run2.txt', guess_times2, fmt=form, delimiter=delim)

    guess_reached_times1 = frame.loc[frame["trial_type"]=="Guess","guess_reached.started"] - triggers[0]
    guess_reached_times2 = frame.loc[frame["trial_type"]=="Guess","guess_reached.started"] - triggers[1]
    guess_reached_times1 = guess_reached_times1.values[guess_reached_times2.values < 0]
    guess_reached_times2 = guess_reached_times2.values[guess_reached_times2.values >= 0]
    guess_reached_times1 = np.array([guess_reached_times1, np.ones(len(guess_reached_times1)) * 1.5, np.ones(len(guess_reached_times1))]).T
    guess_reached_times2 = np.array([guess_reached_times2, np.ones(len(guess_reached_times2)) * 1.5, np.ones(len(guess_reached_times2))]).T
    np.savetxt('onset_files/' + PID + '_guess_reached_onset_run1.txt', guess_reached_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_guess_reached_onset_run2.txt', guess_reached_times2, fmt=form, delimiter=delim)

    # Guess VS Read
    encode_guess_times1 = frame.loc[frame["trial_type"]=="Guess","encode.started"] - triggers[0]
    encode_guess_times2 = frame.loc[frame["trial_type"]=="Guess","encode.started"] - triggers[1]
    encode_guess_times1 = encode_guess_times1.values[encode_guess_times2.values < 0]
    encode_guess_times2 = encode_guess_times2.values[encode_guess_times2.values >= 0]
    encode_guess_times1 = np.array([encode_guess_times1, np.ones(len(encode_guess_times1)) * 3, np.ones(len(encode_guess_times1))]).T
    encode_guess_times2 = np.array([encode_guess_times2, np.ones(len(encode_guess_times2)) * 3, np.ones(len(encode_guess_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_onset_run1.txt', encode_guess_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_onset_run2.txt', encode_guess_times2, fmt=form, delimiter=delim)

    encode_read_times1 = frame.loc[frame["trial_type"]=="Read","encode.started"] - triggers[0]
    encode_read_times2 = frame.loc[frame["trial_type"]=="Read","encode.started"] - triggers[1]
    encode_read_times1 = encode_read_times1.values[encode_read_times2.values < 0]
    encode_read_times2 = encode_read_times2.values[encode_read_times2.values >= 0]
    encode_read_times1 = np.array([encode_read_times1, np.ones(len(encode_read_times1)) * 3, np.ones(len(encode_read_times1))]).T
    encode_read_times2 = np.array([encode_read_times2, np.ones(len(encode_read_times2)) * 3, np.ones(len(encode_read_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_onset_run1.txt', encode_read_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_onset_run2.txt', encode_read_times2, fmt=form, delimiter=delim)

    # Subsequent memory
    # Guess
    encode_guess_correct_times1 = frame.loc[((frame["trial_type"]=="Guess") & (frame["recalled"]==1)),"encode.started"] - triggers[0]
    encode_guess_correct_times2 = frame.loc[((frame["trial_type"]=="Guess") & (frame["recalled"]==1)),"encode.started"] - triggers[1]
    encode_guess_correct_times1 = encode_guess_correct_times1.values[encode_guess_correct_times2.values < 0]
    encode_guess_correct_times2 = encode_guess_correct_times2.values[encode_guess_correct_times2.values >= 0]
    encode_guess_correct_times1 = np.array([encode_guess_correct_times1, np.ones(len(encode_guess_correct_times1)) * 3, np.ones(len(encode_guess_correct_times1))]).T
    encode_guess_correct_times2 = np.array([encode_guess_correct_times2, np.ones(len(encode_guess_correct_times2)) * 3, np.ones(len(encode_guess_correct_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_correct_onset_run1.txt', encode_guess_correct_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_correct_onset_run2.txt', encode_guess_correct_times2, fmt=form, delimiter=delim)

    encode_guess_incorrect_times1 = frame.loc[((frame["trial_type"]=="Guess") & (frame["recalled"]!=1)),"encode.started"] - triggers[0]
    encode_guess_incorrect_times2 = frame.loc[((frame["trial_type"]=="Guess") & (frame["recalled"]!=1)),"encode.started"] - triggers[1]
    encode_guess_incorrect_times1 = encode_guess_incorrect_times1.values[encode_guess_incorrect_times2.values < 0]
    encode_guess_incorrect_times2 = encode_guess_incorrect_times2.values[encode_guess_incorrect_times2.values >= 0]
    encode_guess_incorrect_times1 = np.array([encode_guess_incorrect_times1, np.ones(len(encode_guess_incorrect_times1)) * 3, np.ones(len(encode_guess_incorrect_times1))]).T
    encode_guess_incorrect_times2 = np.array([encode_guess_incorrect_times2, np.ones(len(encode_guess_incorrect_times2)) * 3, np.ones(len(encode_guess_incorrect_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_guess_incorrect_onset_run1.txt', encode_guess_incorrect_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_guess_incorrect_onset_run2.txt', encode_guess_incorrect_times2, fmt=form, delimiter=delim)

    # Read
    encode_read_correct_times1 = frame.loc[((frame["trial_type"]=="Read") & (frame["recalled"]==1)),"encode.started"] - triggers[0]
    encode_read_correct_times2 = frame.loc[((frame["trial_type"]=="Read") & (frame["recalled"]==1)),"encode.started"] - triggers[1]
    encode_read_correct_times1 = encode_read_correct_times1.values[encode_read_correct_times2.values < 0]
    encode_read_correct_times2 = encode_read_correct_times2.values[encode_read_correct_times2.values >= 0]
    encode_read_correct_times1 = np.array([encode_read_correct_times1, np.ones(len(encode_read_correct_times1)) * 3, np.ones(len(encode_read_correct_times1))]).T
    encode_read_correct_times2 = np.array([encode_read_correct_times2, np.ones(len(encode_read_correct_times2)) * 3, np.ones(len(encode_read_correct_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_correct_onset_run1.txt', encode_read_correct_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_correct_onset_run2.txt', encode_read_correct_times2, fmt=form, delimiter=delim)

    encode_read_incorrect_times1 = frame.loc[((frame["trial_type"]=="Read") & (frame["recalled"]!=1)),"encode.started"] - triggers[0]
    encode_read_incorrect_times2 = frame.loc[((frame["trial_type"]=="Read") & (frame["recalled"]!=1)),"encode.started"] - triggers[1]
    encode_read_incorrect_times1 = encode_read_incorrect_times1.values[encode_read_incorrect_times2.values < 0]
    encode_read_incorrect_times2 = encode_read_incorrect_times2.values[encode_read_incorrect_times2.values >= 0]
    encode_read_incorrect_times1 = np.array([encode_read_incorrect_times1, np.ones(len(encode_read_incorrect_times1)) * 3, np.ones(len(encode_read_incorrect_times1))]).T
    encode_read_incorrect_times2 = np.array([encode_read_incorrect_times2, np.ones(len(encode_read_incorrect_times2)) * 3, np.ones(len(encode_read_incorrect_times2))]).T
    np.savetxt('onset_files/' + PID + '_encode_read_incorrect_onset_run1.txt', encode_read_incorrect_times1, fmt=form, delimiter=delim)
    np.savetxt('onset_files/' + PID + '_encode_read_incorrect_onset_run2.txt', encode_read_incorrect_times2, fmt=form, delimiter=delim)